In [1]:
import os
os.chdir("C:\\Users\\hoped\\Desktop\\6.793\\project\\vascular-llm-qa")
print(os.getcwd())
mcq_fname = "C:\\Users\\hoped\\Desktop\\6.793\\project\\vascular-llm-qa\\data\\mcq.csv"


C:\Users\hoped\Desktop\6.793\project\vascular-llm-qa


In [2]:
import pandas as pd
import numpy as np
import pickle
import csv
import os
import time

In [3]:
questions_df = pd.read_csv(mcq_fname, encoding='utf-8')
questions_df

,question_id,source,labels,topic,question,answer,explanation
0,1a,apdvs,"cs, lq",carotid disorders,A 75-year-old male smoke presents with recent ...,D,Right carotid endarterectomy. The patient has ...
1,1b,apdvs,"cs, sq",carotid disorders,A 75-year-old man had temporary vision loss in...,D,Right carotid endarterectomy. The patient has ...
2,2a,apdvs,"gk, lq",carotid disorders,What is the first muscle layer encountered in...,C,The platysma is the most superficial muscle in...
3,2b,apdvs,"gk, sq",carotid disorders,"During a carotid endarterectomy, which neck mu...",C,The platysma is the most superficial muscle in...
4,3a,apdvs,"gk, lq",lower extremity arterial disease,A 65-year old male with a history of hyperten...,B,Claudication is defined as reproducible pain o...
...,...,...,...,...,...,...,...
446,56.ten,case_studies,cs,venuous disease,A 19 year old female was brought in to the Eme...,C,NaN
447,57.2,case_studies,gk,venuous disease,Which of the following statements regarding th...,"B, C, E",NaN
448,57.3,case_studies,gk,venuous disease,Which of the following statements regarding th...,"A, B, D",NaN
449,57.4,case_studies,gk,venuous disease,What are the long-term results of derivative a...,"A, C",NaN


In [5]:
# do data conversion
import json
import re
from utils.data_utils import extract_answer_choices
#from retriever.retrieval import retrieve_docs
from tqdm import tqdm

import os
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import json
import random

# chunk size 500 stuff
# Load FAISS index and corpus (assumes pre-built)
# faiss_index = faiss.read_index("retriever/faiss_index/corpus.index")
# with open("retriever/faiss_index/metadata.json") as f:
#     metadata = json.load(f)


faiss_index = faiss.read_index("retriever/faiss_index/corpus_100.index")
with open("retriever/faiss_index/metadata_100.json") as f:
    metadata = json.load(f)

embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def get_chunk_by_index(idx, chunk_dir="retriever/corpus_100"): # change for 100
    """Given a FAISS index, load the corresponding chunk text."""
    filename = metadata[idx]["filename"]
    path = os.path.join(chunk_dir, filename)
    with open(path, "r", encoding="utf-8") as f: # CHANGED to fix unicode error
        return f.read()

# next; include golden for training
def retrieve_docs(question, include_relevant=True, top_k=4):
    """Retrieve documents for a question.

    If include_relevant=True: return top-k FAISS retrieved docs.
    If include_relevant=False: return true distractors, sampled randomly while excluding top matches.
    """

    # Embed the question
    question_embedding = embedder.encode([question])
    question_embedding = np.array(question_embedding).reshape(1, -1)

    # Search FAISS index
    _, indices = faiss_index.search(question_embedding, top_k)
    retrieved_indices = list(indices[0])

    if include_relevant:
        # Return the top-k most similar documents
        return [get_chunk_by_index(idx) for idx in retrieved_indices[:top_k]]
    else:
        # Build candidate pool by excluding top-k retrieved
        all_indices = set(range(len(metadata)))
        blocked_indices = set(retrieved_indices[:top_k])
        candidate_indices = list(all_indices - blocked_indices)

        # Randomly sample true distractors
        distractor_indices = random.sample(candidate_indices, top_k)
        return [get_chunk_by_index(idx) for idx in distractor_indices]




def convert_df_to_json(df):
    id_question = {}
    for _, row in tqdm(df.iterrows()):
        full_question_text = row["question"]
        try:
            question_stem, answer_choices = extract_answer_choices(full_question_text)
        except Exception as e:
            print(f"Error parsing choices for question: {full_question_text}")
            raise e

        raw_answers = str(row['answer'])
        answer_letters = re.split(r"[,\s]+", raw_answers.strip())
        letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        for a in answer_letters:
          i = letters.index(a)
          try:
            answer_choices[i]
          except Exception as e:
            print(a, i)
            print(row["question_id"])
            print("\n".join(answer_choices))
            raise e

          # make sure that all answers exist
        full_answers = [f'{letters[i]}. {x}' for i,x in enumerate(answer_choices) if letters[i] in answer_letters]

        relevant_docs = retrieve_docs(question_stem)

        example = {
            "question_id": row["question_id"],
            "question": question_stem,
            "choices": answer_choices,
            "correct_answer": answer_letters,
            "full_answers": full_answers,
            "explanation": row.get("explanation", ""),
            "source": row.get("source", ""),
            "tags": row.get("labels", "").split(","),
            "topic": row.get("topic", ""),
            "context": relevant_docs,
        }
        id_question[row["question_id"]] = example

    return id_question

id_question = convert_df_to_json(questions_df)

id_question["5a"]

451it [00:34, 13.03it/s]


{'question_id': '5a',
 'question': 'A 56-year-old male with hypertension, diabetes presents to the emergency department with redness and purulence draining from his right great toe. His heart rate is 110 beats/min, his blood pressure is 100/80 mm Hg, and his temperature is 38.5ºC. Laboratory tests show a white blood count of 14,000/µL. An x-ray of his foot is performed, which demonstrates gas in his right great toe. What is the best course of treatment?',
 'choices': ['Outpatient treatment with oral antibiotics and dressing changes.',
  'Inpatient treatment with intravenous antibiotics alone.',
  'Inpatient treatment with intravenous antibiotics and dressing changes.',
  'Inpatient treatment with intravenous antibiotics and emergent surgical debridement .'],
 'correct_answer': ['D'],
 'full_answers': ['D. Inpatient treatment with intravenous antibiotics and emergent surgical debridement .'],
 'explanation': 'Inpatient treatment with intravenous antibiotics and emergent surgical debride

In [7]:
import pickle 
with open("question_context_100.pickle", "wb") as f:
    pickle.dump(id_question, f)

In [9]:
os.getcwd()


'C:\\Users\\hoped\\Desktop\\6.793\\project\\vascular-llm-qa'